In [1]:
!pip install -qqqU sagemaker wandb datasets transformers

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
distributed 2022.7.0 requires tornado<6.2,>=6.0.3, but you have tornado 6.3.3 which is incompatible.


In [3]:
import wandb
run = wandb.init()
artifact = run.use_artifact('parambharat/wandbot_eval/eval_results:latest', type='dataset')
artifact_dir = artifact.download()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: capecape. Use `wandb login --relogin` to force relogin


wandb:   1 of 1 files downloaded.  


In [4]:
table = artifact.get("eval_results")

wandb:   1 of 1 files downloaded.  


In [5]:
table.columns

['question',
 'answer',
 'reaction',
 'contexts',
 'answer_relevancy',
 'context_relavency',
 'faithfulness',
 'correctness']

In [6]:
import pandas as pd

df = pd.DataFrame(table.data, columns=table.columns)

In [7]:
df = df[df.reaction!=-1]

In [8]:
df.head()

,question,answer,reaction,contexts,answer_relevancy,context_relavency,faithfulness,correctness
0,"write sample code to upload a table to a run, ...","Sure, here is how you can upload a table to a ...",0,[description: Explore how to use W&B Tables wi...,0.897,0.0,0.333333,1
1,"write sample code to upload a table to a run, ...","Sure, here is how you can upload a table to a ...",0,[description: How to use the sidebar and table...,0.884,0.0,1.000000,1
2,is there an integration with MONAI?,"Hmm, I'm not sure. The provided documentation ...",0,[Integrations\n\nView source on GitHub\n\nModu...,0.884,0.0,0.000000,1
4,how do I resume a run and attach an artifact t...,**Warning: Falling back to gpt-3.5.** These re...,0,[```\n\nResume Guidance\n\nThere are different...,0.911,0.0,0.500000,1
5,can i attach an artifact to an already existin...,"Yes, you can attach an artifact to an already ...",0,[```\n\n✔️ run.use_artifact\n\nThese steps are...,0.850,0.0,0.000000,1


In [9]:
len(df)

811

In [10]:
len(df.iloc[245]["contexts"])

8

In [11]:
def prompt_template(row):
    return ("You are an AI assistant designed to assist developers with everyday tasks related to Weight & Biases "
            "and provide helpful information. As an expert in the open-source python SDK wandb answer the following"
            "question based on the context. Answer in formatted Markdown.\n"
            "Write a response that appropriately answer the question\n\n"
            "### Question:\n{question}\n\n### Response:\n{answer}").format_map(row)


In [12]:
prompt_template(df.iloc[0])

'You are an AI assistant designed to assist developers with everyday tasks related to Weight & Biases and provide helpful information. As an expert in the open-source python SDK wandb answer the followingquestion based on the context. Answer in formatted Markdown.\nWrite a response that appropriately answer the question\n\n### Question:\nwrite sample code to upload a table to a run, then download that table again in a new run as a Pandas Dataframe\n\n### Response:\nSure, here is how you can upload a table to a run and then download it in a new run as a Pandas DataFrame:\n\nFirst, let\'s create a table and log it to a run:\n\n```python\nimport wandb\n\n# Initialize a new run\nrun = wandb.init(project="table-test")\nmy_table = wandb.Table(\n    columns=["a", "b"], \n    data=[["a1", "b1"], ["a2", "b2"]]\n    )\nrun.log({"Table Name": my_table})\n```\n\nNow, let\'s download the table in a new run:\n\n```python\nimport wandb\n\n# Initialize a new run\nrun = wandb.init(project="table-test")

## Prepare the Dataset

If you use CodeLLama we need to format the instructions accordingly!

In [13]:
def merge_contexts(ctxs):
    "The model context is too short!!!"
    return ctxs[0]  # too long!
    # return "\n".join(ctxs)

In [14]:
B_INST, E_INST = "[INST] ", " [/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
EOS = "</s>"


def format_text(row):
    text = (
        B_INST
        + B_SYS
        + "You are an AI assistant designed to assist developers with everyday tasks related to Weight & Biases"
        + "and provide helpful information. As an expert in the open-source python SDK wandb answer the following"
        # + "question based on the context below. Answer in formatted Markdown.\n"
        + "question below. Answer in formatted Markdown.\n"
        # + merge_contexts(row["contexts"])
        + E_SYS
        + row["question"]
        + E_INST
        + "\n[W&B]\n"
        + row["answer"]
        + "\n[/W&B]"
        + EOS
    )

    return text

In [15]:
one_example = format_text(df.iloc[0])
print(one_example)

[INST] <<SYS>>
You are an AI assistant designed to assist developers with everyday tasks related to Weight & Biasesand provide helpful information. As an expert in the open-source python SDK wandb answer the followingquestion below. Answer in formatted Markdown.

<</SYS>>

write sample code to upload a table to a run, then download that table again in a new run as a Pandas Dataframe [/INST]
[W&B]
Sure, here is how you can upload a table to a run and then download it in a new run as a Pandas DataFrame:

First, let's create a table and log it to a run:

```python
import wandb

# Initialize a new run
run = wandb.init(project="table-test")
my_table = wandb.Table(
    columns=["a", "b"], 
    data=[["a1", "b1"], ["a2", "b2"]]
    )
run.log({"Table Name": my_table})
```

Now, let's download the table in a new run:

```python
import wandb

# Initialize a new run
run = wandb.init(project="table-test")

# Use the artifact API to access the table
artifact = run.use_artifact("run-<run-id>-Table N

In [16]:
dataset = []
for index, row in df.iterrows():
    dataset.append({"text": format_text(row)})

In [17]:
import json
def save_jsonl(data, filename):
    with open(filename, 'w') as file:
        for entry in data:
            json.dump(entry, file)
            file.write('\n')

In [18]:
save_jsonl(dataset, "wandb_questions_ds.jsonl")

## SageMaker auth

In [19]:
import sagemaker
import boto3
sess = sagemaker.Session()
# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it not exists
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker role arn: arn:aws:iam::372108735839:role/SageMakerExecutionRole
sagemaker bucket: sagemaker-us-east-1-372108735839
sagemaker session region: us-east-1


## Preparing a dataset

In [20]:
MODEL_NAME = "codellama/CodeLlama-7b-Instruct-hf"

In [21]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_auth_token=True)
tokenizer.pad_token = tokenizer.eos_token

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
/opt/conda/lib/python3.10/site-packages/transformers/models/auto/tokenization_auto.py:655: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


In [22]:
from datasets import load_dataset

In [23]:
dataset = load_dataset(path=".", data_files="wandb_questions_ds.jsonl", split="train")
dataset

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['text'],
    num_rows: 811
})

In [24]:
dataset[0]

{'text': '[INST] <<SYS>>\nYou are an AI assistant designed to assist developers with everyday tasks related to Weight & Biasesand provide helpful information. As an expert in the open-source python SDK wandb answer the followingquestion below. Answer in formatted Markdown.\n\n<</SYS>>\n\nwrite sample code to upload a table to a run, then download that table again in a new run as a Pandas Dataframe [/INST]\n[W&B]\nSure, here is how you can upload a table to a run and then download it in a new run as a Pandas DataFrame:\n\nFirst, let\'s create a table and log it to a run:\n\n```python\nimport wandb\n\n# Initialize a new run\nrun = wandb.init(project="table-test")\nmy_table = wandb.Table(\n    columns=["a", "b"], \n    data=[["a1", "b1"], ["a2", "b2"]]\n    )\nrun.log({"Table Name": my_table})\n```\n\nNow, let\'s download the table in a new run:\n\n```python\nimport wandb\n\n# Initialize a new run\nrun = wandb.init(project="table-test")\n\n# Use the artifact API to access the table\nartif

We define some helper functions to pack our samples into sequences of a given length and then tokenize them.tokenizer

In [25]:
from random import randint
from itertools import chain
from functools import partial


# template dataset to add prompt to each sample
def template_dataset(sample):
    sample["text"] = f"{sample}{tokenizer.eos_token}"
    return sample


# apply prompt template per sample
dataset = dataset.map(template_dataset)
# print random sample
print(dataset[randint(0, len(dataset))]["text"])

# empty list to save remainder from batches to use in next batch
remainder = {"input_ids": [], "attention_mask": [], "token_type_ids": []}

Map:   0%|          | 0/811 [00:00<?, ? examples/s]

{'text': '[INST] <<SYS>>\nYou are an AI assistant designed to assist developers with everyday tasks related to Weight & Biasesand provide helpful information. As an expert in the open-source python SDK wandb answer the followingquestion below. Answer in formatted Markdown.\n\n<</SYS>>\n\nwhere can I find info about my weights & biases license? [/INST]\n[W&B]\nYou can find information about your Weights & Biases account, including storage usage, in your user settings. To access your user settings, navigate to your user profile page and select your user icon on the top right corner. From the dropdown, choose "Settings."\n\nFor information about storage and pricing, you can visit the [Pricing page](<https://wandb.ai/site/pricing>).\n\nPlease note that I cannot provide a direct link to your user settings, as it requires you to be logged in to your Weights & Biases account.\n\n*References*:\n\n>- <https://docs.wandb.ai/guides/app/settings-page/user-settings>\n>- <https://docs.wandb.ai/guide

In [26]:
def chunk(sample, chunk_length=1024):
    # define global remainder variable to save remainder from batches to use in next batch
    global remainder
    # Concatenate all texts and add remainder from previous batch
    concatenated_examples = {k: list(chain(*sample[k])) for k in sample.keys()}
    concatenated_examples = {k: remainder[k] + concatenated_examples[k] for k in concatenated_examples.keys()}
    # get total number of tokens for batch
    batch_total_length = len(concatenated_examples[list(sample.keys())[0]])

    # get max number of chunks for batch
    if batch_total_length >= chunk_length:
        batch_chunk_length = (batch_total_length // chunk_length) * chunk_length

    # Split by chunks of max_len.
    result = {
        k: [t[i : i + chunk_length] for i in range(0, batch_chunk_length, chunk_length)]
        for k, t in concatenated_examples.items()
    }
    # add remainder to global variable for next batch
    remainder = {k: concatenated_examples[k][batch_chunk_length:] for k in concatenated_examples.keys()}
    # prepare labels
    result["labels"] = result["input_ids"].copy()
    return result


# tokenize and chunk dataset
lm_dataset = dataset.map(
    lambda sample: tokenizer(sample["text"]), batched=True, remove_columns=list(dataset.features)
).map(
    partial(chunk, chunk_length=1024),
    batched=True,
)

# Print total number of samples
print(f"Total number of samples: {len(lm_dataset)}")

Map:   0%|          | 0/811 [00:00<?, ? examples/s]

Map:   0%|          | 0/811 [00:00<?, ? examples/s]

Total number of samples: 304


Pretty small =P

## Save to a bucket and W&B

In [27]:
# save train_dataset to s3
training_input_path = f's3://{sess.default_bucket()}/processed/wandbot/train'
lm_dataset.save_to_disk(training_input_path)

print("uploaded data to:")
print(f"training dataset to: {training_input_path}")

Saving the dataset (0/1 shards):   0%|          | 0/304 [00:00<?, ? examples/s]

uploaded data to:
training dataset to: s3://sagemaker-us-east-1-372108735839/processed/wandbot/train


In [28]:
at = wandb.Artifact(
    name="wandbot_dataset", 
    type="dataset",
    description="A wandbot dataset of questions and answers about W&B")

In [29]:
with wandb.init(project="aws_llm_demo"):
    at.add_reference(training_input_path)
    wandb.log_artifact(at)

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


wandb: Generating checksum for up to 10000 objects in "sagemaker-us-east-1-372108735839/processed/wandbot/train"... Done. 0.1s


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


## Train a model

We need to save our W&B credentials on the folder that will be copied to the SM instance

In [30]:
wandb.sagemaker_auth(path="scripts")

In [31]:
import transformers

In [32]:
transformers.__version__

'4.34.0'

In [33]:
import time
from sagemaker.huggingface import HuggingFace
from huggingface_hub import HfFolder

# define Training Job Name 
job_name = f'huggingface-qlora-codellama7'

lr = 2e-4

# hyperparameters, which are passed into the training job
hyperparameters ={
    'model_id': MODEL_NAME,                             # pre-trained model
    'dataset_path': '/opt/ml/input/data/training',    # path where sagemaker will save training dataset
    'epochs': 1,                                      # number of training epochs
    'per_device_train_batch_size': 2,                 # batch size for training
    'lr': lr,                                       # learning rate used during training
    'hf_token': HfFolder.get_token(),                 # huggingface token to access llama 2
    'merge_weights': True,                            # wether to merge LoRA into the model (needs more memory)
    'report_to': "wandb",                              # report to wandb
    'wandb_project': "aws_llm_demo",
    "run_name":  f"{MODEL_NAME}__qlora",
}
    
# create the Estimator
huggingface_estimator = HuggingFace(
    entry_point          = 'run_clm.py',      # train script
    source_dir           = 'scripts',         # directory which includes all the files needed for training
    instance_type        = 'ml.g5.4xlarge',   # instances type used for the training job
    instance_count       = 1,                 # the number of instances used for training
    base_job_name        = job_name,          # the name of the training job
    role                 = role,              # Iam role used in training job to access AWS ressources, e.g. S3
    volume_size          = 300,               # the size of the EBS volume in GB
    transformers_version = '4.28',            # the transformers version used in the training job
    pytorch_version      = '2.0',             # the pytorch_version version used in the training job
    py_version           = 'py310',           # the python version used in the training job
    hyperparameters      =  hyperparameters,  # the hyperparameters passed to the training job
    environment          = { "HUGGINGFACE_HUB_CACHE": "/tmp/.cache"}, # set env variable to cache models in /tmp
)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [34]:
# define a data input dictonary with our uploaded s3 uris
data = {'training': training_input_path}

# starting the train job with our uploaded datasets as input
huggingface_estimator.fit(data, wait=True)

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.


Using provided s3_resource


ValueError: Unsupported huggingface version: 4.34. You may need to upgrade your SDK version (pip install -U sagemaker) for newer huggingface versions. Supported huggingface version(s): 4.4.2, 4.5.0, 4.6.1, 4.10.2, 4.11.0, 4.12.3, 4.17.0, 4.26.0, 4.28.1, 4.4, 4.5, 4.6, 4.10, 4.11, 4.12, 4.17, 4.26, 4.28.